In [1]:
# https://microsoft.github.io/sql-ml-tutorials/python/rentalprediction/step/2.html

In [2]:
import argparse
import pandas
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
#BOT_FILE='csv/bot_100k.csv'
BOT_FILE='csv/bot_2m.csv'
# HUM_FILE='csv/hum_100k.csv'
HUM_FILE='csv/hum_2m.csv'
# HUM_FILE='csv/human.csv'

In [4]:
bot_dataframe = pandas.read_csv(BOT_FILE, header=None, error_bad_lines=False, warn_bad_lines=False)
hum_dataframe = pandas.read_csv(HUM_FILE, header=None, error_bad_lines=False, warn_bad_lines=False)

/Users/rain/git/bot-data/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
bot_dataframe['target'] = 1
hum_dataframe['target'] = 0

In [6]:
dataset = pandas.concat([bot_dataframe, hum_dataframe]).sample(frac=1).reset_index(drop=True)
dataset

,0,1,2,3,4,5,6,7,8,9,10,target
0,106.12.152.38,NaN,NaN,2020-09-28 14:31:16,POST,/jing.php,HTTP/1.1,405.0,166,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,1
1,158.69.5.181,NaN,NaN,2018-04-05 02:30:52,POST,/administrator/index.php,HTTP/1.1,200.0,4498,NaN,NaN,0
2,106.54.120.49,NaN,NaN,2020-10-13 11:58:22,GET,/images/up.php,HTTP/1.1,200.0,474,NaN,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,1
3,111.230.228.235,NaN,NaN,2020-10-12 00:48:36,POST,/sanan.php,HTTP/1.1,405.0,568,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,1
4,103.67.235.109,NaN,NaN,2017-11-21 20:20:31,GET,/scripts/popup-pomo.php,HTTP/1.1,404.0,220,http://site.ru,Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3920072,45.10.172.12,NaN,NaN,2020-09-29 14:41:37,GET,/phpMyadmi/index.php,HTTP/1.1,200.0,474,NaN,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1
3920073,118.25.53.11,NaN,NaN,2020-10-13 03:17:15,GET,/dbadmin/index.php,HTTP/1.1,200.0,474,NaN,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,1
3920074,86.235.205.166,NaN,NaN,2020-10-05 19:53:41,GET,/phpmyadmin4/index.php?lang=en,HTTP/1.1,200.0,474,NaN,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1
3920075,118.25.53.11,NaN,NaN,2020-10-13 03:14:16,POST,/aaaaaa1.php,HTTP/1.1,405.0,166,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:45...,1


In [7]:
dataset_clean = dataset.fillna(0)
dataset_clean = dataset_clean[dataset_clean[3].str.len() == 19]  # Remove invalid date strings

dataset_clean[0] = pandas.Categorical(dataset_clean[0]).codes
dataset_clean[1] = pandas.Categorical(dataset_clean[1]).codes
dataset_clean[2] = pandas.Categorical(dataset_clean[2]).codes
dataset_clean[3] = pandas.to_datetime(dataset_clean[3]).astype(int)/1_000_000_000

dataset_clean[4] = pandas.Categorical(dataset_clean[4]).codes
dataset_clean[5] = pandas.Categorical(dataset_clean[5]).codes
dataset_clean[6] = pandas.Categorical(dataset_clean[6]).codes
dataset_clean[7] = dataset_clean[7].astype(int)
dataset_clean[8] = pandas.to_numeric(dataset_clean[8], errors='coerce').notnull()
dataset_clean[9] = pandas.Categorical(dataset_clean[9]).codes
dataset_clean[10] = pandas.Categorical(dataset_clean[10]).codes
dataset_clean

,0,1,2,3,4,5,6,7,8,9,10,target
0,2858,0,0,1.601303e+09,5,68612,8,405,True,0,11380,1
1,16551,0,0,1.522895e+09,5,7811,8,200,True,0,0,0
2,3247,0,0,1.602590e+09,2,39637,8,200,True,0,8257,1
3,6027,0,0,1.602464e+09,5,83407,8,405,True,0,8849,1
4,1484,0,0,1.511296e+09,2,83467,8,404,True,8035,10558,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3920072,57497,0,0,1.601390e+09,2,81935,8,200,True,0,6230,1
3920073,8247,0,0,1.602559e+09,2,21461,8,200,True,0,621,1
3920074,79487,0,0,1.601928e+09,2,82076,8,200,True,0,6447,1
3920075,8247,0,0,1.602559e+09,5,6371,8,405,True,0,9101,1


In [8]:
dataset_clean.dtypes

0           int32
1            int8
2            int8
3         float64
4            int8
5           int32
6            int8
7           int64
8            bool
9           int16
10          int16
target      int64
dtype: object

In [9]:
train_df = dataset_clean.sample(frac=0.8, random_state=1)
test_df = dataset_clean.loc[~dataset_clean.index.isin(train_df.index)]
columns = [c for c in train_df.columns.tolist() if c != 'target']

In [10]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
linear_model = LinearRegression()
linear_model.fit(train_df[columns], train_df['target'])

LinearRegression()

In [11]:
# This is a list of predictions based on the input columns from our test data set
predictions = linear_model.predict(test_df[columns])
predictions

array([0.10341694, 0.86599869, 0.94891672, ..., 0.890752  , 0.7275354 ,
       0.86643035])

In [12]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
# This tells you how accurate the model is (how far off it is in its predictions "on average")
mse = mean_squared_error(predictions, test_df['target'])
mse

0.08864102973805614